<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/POI-point-of-interest-case-study-part1-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Case Study: Accessibility - Part 1


The City of Columbus has put together a collection of points of interest for different functional categories. This data can be viewed at their interactive website https://opendata.columbus.gov/datasets/columbus::points-of-interest/explore. The description of the data can be found at https://maps2.columbus.gov/arcgis/rest/services/Schemas/PointsOfInterest/MapServer/10. The raw geojson file has been posted at https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson. We will use this geojson file in this tutorial.

The goal of this tutorial is to demonstrate the use of Python for spatial data analysis. More specifically, we will examine how residents in Franklin county can access some of the services represented in this data set. Here, we must realize that the data is incomplete because it may not fully cover areas outside the city of Columbus and some points of interest may not be included in the data either. The main purpose is the demonstrate the use of data and Python coding to analyze accessibility. 

Most of the code is not directly shown in the original tutorial. The associated video (from the in-class demo) will have a walk-through of most of the processes. After finishing this tutorial, students should answer the following questions (with code):

1. How many types of points of interest are represented in the data and how many instances are there in each type?
2. Where are the POIs distributed on a map? 
3. If we redistribute the population of Franklin County evenly into cells in each census block group, who does the population distribution look like on the map?
4. How accessibility to a certain service (medical) looks like on the map?
5. What are the shortcomings of the analysis presented in this tutorial?


## Accessibility: a very short introduction

We will use this data to compute the accessibility to various services and we will use health as an example. Accessibility is a big topic and we will focus on a particular approach called the two-step floating catchment area (2SFCA) method (Luo and Wang 2003). Let's assume we want to compute the accessibility at location $i$ to various services provided at locations denoted as $j$. We call $i$ the demand location and $j$ the supply location. The first step is to compute the supply to demand ratio at each supply location $j$ as follows:

$R_j = \Large\frac{S_j}{\sum_{k \in (d_{jk} < d_0)} D_k}$

where $R_j$ is the ratio, $S_j$ is the capacity of the supply at location $j$, $d_{kj}$ is the demand from location $k$ that require supply from $j$, and $d_0$ is a distance buffer that defines the catchment of supply $j$. We use $(d_{jk} < d_0)$ to a set of locations ($k$) whose distance to $j$ is smaller than $d_0$. We can replace this set with any set that can be used to define the catchment and therefore the catchment doesn't have to be a circular buffer but can be of any shape. The above equation basically counts all the demand within the catchment (defined by a radius of $d_0$) and split the supply among them. Here, the demand is weighted equally regardless of how far it is from the supply. It is reasonable, as many researchers have done so, to further weight the demand by some kind of  impedance function (e.g., distance decay function). This, however, is dependent on the actual demand and supply. 

The second step, now that we have worked out the supply side, is to actually compute the accessibility at the demand side by simply add up all the supply to demand ratios that are within the catchment radius of the demand location:

$\Large A_i = \sum_{j\in (d_{ij}<d_0)}{R_j}$

where $A_i$ is the accessibility at demand $i$ and we use $(d_{ij} < d_0)$ to denote the supply locations ($j$) that are within the distance of $d_0$. 

In an application, we have many demand locations to calculate and we will "float" this calculation over the space so that we get the accessibility measure of all the locations of our interest. 

**Work cited**

Luo, W., & Wang, F. (2003). Measures of spatial accessibility to health care in a GIS environment: synthesis and a case study in the Chicago region. Environment and planning B: planning and design, 30(6), 865-884.

In [ ]:
!rm -rf geom
!git clone https://github.com/gisalgs/geom.git

In [ ]:
import urllib.request as request
import json 

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/columbus_points_of_interest.geojson'
with request.urlopen(url) as response:
    poi = json.loads(response.read())

len(poi['features'])

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/blockgrps_pop_franklin_2.geojson'
with request.urlopen(url) as response:
    blkgrps = json.loads(response.read())

len(blkgrps['features'])

In [ ]:
blkgrps['features'][0]['properties'].keys()

In [ ]:
# check how many are multiplygons -- all of them!

sum([f['geometry']['type'] == 'MultiPolygon' for f in blkgrps['features']])

In [ ]:
# How many have multiple parts

# there are two 

sum([len(f['geometry']['coordinates'])>1 for f in blkgrps['features']])

In [ ]:
# holes?

# How many have holes -- forget about the two with multiple parts

# there is one

sum([len(f['geometry']['coordinates'][0])>1 for f in blkgrps['features']])

In [ ]:
for f in blkgrps['features']:
    if len(f['geometry']['coordinates'][0])>1:
        print(f['properties']['TRACT'], f['properties']['BLKGRP'])

Explore the types of geometry in the POI data.

In [ ]:
geom_types = []
for p in poi['features']:
    t = p['geometry']['type']
    if t not in geom_types:
        geom_types.append(t)

geom_types

Now we know all the features are of the same type: Point. Let's know explore the attributes a little bit before we draw the map.

In [ ]:
p['properties']

In [ ]:
poi_types = {}
for p in poi['features']:
    t = p['properties']['POI_TYPE']
    if t not in poi_types:
        poi_types[t] = 1
    else:
        poi_types[t] += 1

print(poi_types)

In [ ]:
poi_types_main = {}
for t in poi_types:
    tt = t.split(' - ')[0]
    if tt not in poi_types_main:
        poi_types_main[tt] = poi_types[t]
    else:
        poi_types_main[tt] += poi_types[t]

poi_types_main

Something is wrong here. Apparently, some dashes in the key are not the same dashes. We can suspect they are automatically converted from a hyphen symbol from the keyboard to a dash when people typed the values in their computer system (a lot of times doing things in Microsoft Word does this if we don't pay attention). Let's give this a further examination:

In [ ]:
for c in 'Retail – Commercial/Retail':
    print(ord(c), end=' ')


The `ord` function returns the Unicode of a character. If the code is smaller than 128, then the character is what we call an ascii character, or in plain English, a text. Otherwise, it is a special character that is coded beyond the 8 bit ascii characters. Guess which on is beyond?

Guilty as charged!

Now we need to find a way to make sure the dash-like symbol is actually used to split the strings. To get the character, we use the `chr` function in an f-string:

In [ ]:
f' {chr(8211)} '

In [ ]:
'Retail – Commercial/Retail'.split(f' {chr(8211)} ')

Now we are going to use multiple delimiters to split strings. Unfortunately, the default method `split` for strings cannot do this and we will have to use something that is bigger than what we need. We are going to use another native Python library called `re`, which is used to handle regular expressions (hence re). This is an extremely complicated topic--people write books about this, one being [here](https://www.amazon.com/Mastering-Regular-Expressions-Jeffrey-Friedl/dp/0596528124/ref=asc_df_0596528124/?tag=hyprod-20&linkCode=df0&hvadid=692875362841&hvpos=&hvnetw=g&hvrand=10753013780408513308&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9014986&hvtargid=pla-2281435177858&psc=1&mcid=86ae5603da983d6b9314bd0e9ed46f65&hvocijid=10753013780408513308-0596528124-&hvexpln=73)--and we will only use one single function in this library. It is also called `split`:

In [ ]:
import re
poi_types_main = {}
for t in poi_types:
    # tt = t.split(' - ')[0]
    tt = re.split(f' - | {chr(8211)} ', t)[0]
    if tt not in poi_types_main:
        poi_types_main[tt] = poi_types[t]
    else:
        poi_types_main[tt] += poi_types[t]

poi_types_main

Now there is only one problem: the two categories called 'Emergency Response' and 'Emergency  Response' should be the same. Again human errors. We will write code to merge them into the right one. We again use a [method in the `re` module called `sub`](https://docs.python.org/3/library/re.html#re.sub). In the following code, the symbol + means a pattern formed by multiple of the string before the symbol, which will be replaced by the second argument.

In [ ]:
re.sub(' +', ' ', 'a    b')

In [ ]:
poi_types_final = {}
for t in poi_types_main:
    tt = re.sub(' +', ' ', t)
    if tt not in poi_types_final:
        poi_types_final[tt] = poi_types_main[t]
    else:
        poi_types_final[tt] += poi_types_main[t]

poi_types_final

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
y_pos = range(len(poi_types_final))

plt.barh(y_pos, poi_types_final.values(), align='center', alpha=0.5)
plt.yticks(y_pos, poi_types_final)
plt.xlabel('Count')
plt.title('POI Types')

plt.show()

This is nice but we would like to show the bars ordered. Unfortunately Python doesn't allow us to sort the dictionary. We will have use a list to do this.

In [ ]:
summary = [ [t, poi_types_final[t]] for t in poi_types_final]
summary.sort(key = lambda x: x[1])

In [ ]:
y_pos = range(len(summary))

plt.barh(y_pos, [v[1] for v in summary], align='center', alpha=0.5)
plt.yticks(y_pos, [v[0] for v in summary])
plt.xlabel('Count')
plt.title('POI Types')

plt.show()

## Color

Before we go any further, let's talk a little more about color in matplotlib. Eventually we want to map these POIs using their categories, and we have a set of category small enough to make a map (so that we don't have to use too many colors). 

A typical way to use color for visualization is to define a color ramp (or color map) and we can then map our numerical data range to that color map, and in this way each value will be automatically assigned a color.

Matplotlib has a huge set of color maps for different purposes: https://matplotlib.org/stable/users/explain/colors/colormaps.html. 

In [ ]:
from matplotlib import colormaps
for c in colormaps:
    print(c, end=' ')

More specifically, there are colors some colors specifically useful for categorical data. But confusingly, they are in a collection called color_sequences. We should not confuse this name with sequential color. They are merely a sequence of color put together for categorical data. We are going to use the one called `tab20`.

In [ ]:
from matplotlib import color_sequences
for c in color_sequences:
    print(c, end=' ')

In [ ]:
plt.get_cmap('tab20')

We can now assign a color to each of the types using a dictionary called `poi_types_color`.

In [ ]:
cmap = color_sequences['tab20']
poi_types_color = {}
i = 0
for t in poi_types_final:
    poi_types_color[t] = cmap[i]
    i += 1

poi_types_color

Now we assign each feature in the POI data a color, determined by its type.

In [ ]:
# set up the data

for f in poi['features']:
    f_type = re.split(f' - | {chr(8211)} ', f['properties']['POI_TYPE'])[0]
    f_type = re.sub(' +', ' ', f_type)
    f['poi_type'] = f_type
    f['color'] = poi_types_color[f_type]

In [ ]:
_, ax = plt.subplots()

for t in poi_types_final:
    x = [f['geometry']['coordinates'][0] for f in poi['features'] if f['poi_type']==t]
    y = [f['geometry']['coordinates'][1] for f in poi['features'] if f['poi_type']==t]
    c = [f['color'] for f in poi['features'] if f['poi_type']==t]
    if x:
        scatter = plt.scatter(x, y, c=c, label=t, marker='o', alpha=0.5, s=5)

ax.set_aspect(1)
ax.legend( loc="right", bbox_to_anchor=(1.5, 0.5),)
plt.show()

Let's just draw a small sample of the points in order to get a clearer picture of things.

In [ ]:
import random
some_poi = random.sample(poi['features'], 1000)

_, ax = plt.subplots()
for t in poi_types_final:
    x = [f['geometry']['coordinates'][0] for f in some_poi if f['poi_type']==t]
    y = [f['geometry']['coordinates'][1] for f in some_poi if f['poi_type']==t]
    c = [f['color'] for f in some_poi if f['poi_type']==t]
    if x:
        scatter = plt.scatter(x, y, c=c, label=t, marker='o', alpha=0.8)

ax.set_aspect(1)
ax.legend( loc="right", bbox_to_anchor=(1.5, 0.5))
plt.show()

## Interpolation: population

Before we can calculate accessibility, it is important to get the demand figured out. Here, we will use the total population to represent the demand. More specifically, we will create a raster data set where each cell has a certain size. To do this, let's first set the area to the range of -83.2 to -82.7 longitudes and 39.8 to 40.2 latitudes. This is a rough estimate using the coordinates of the point and we can come back and change it if the area is off. We want a increment of 0.01 degrees. On a great circle or on a meridian, this increment is equivalent to about 0.69 miles or 1.11 km, as shown below:



In [ ]:
from math import pi

radius = 3959 # miles
circumference = 2 * pi * radius 
degreelength = circumference/360
degreelength, 0.01*degreelength

We use the point at the center of the "cell" to represent the cell and we first get the coordinates of these centers, as `xs` and `ys` below.

In [ ]:
xmin, xmax = -83.2, -82.7
ymin, ymax = 39.8, 40.2
delta = 0.01

xn = int((xmax-xmin)/delta)
yn = int((ymax-ymin)/delta)

xs = [xmin + i*delta + delta/2 for i in range(xn)] # make sure the point is at the center of the cell of 0.05 degree
ys = [ymin + i*delta + delta/2 for i in range(yn)]

len(xs), len(ys)


We interpolate the population data in census block groups into the cells. To do so, we go through each cell and check which block group contains the center. After we are done, we split the population to the cells in the block group. 

But we will have to do this in a "smart" way because we don't want to go over the block groups over and over again. To do so, we get the bounds of the polygons and do things from there. This is going to be useful later on and we develop a function for this purpose.

We are going to use a [specific function that returns the bounds](https://github.com/gisalgs/geom/blob/master/multipolygon_util.py) of a geojson polygon or multipolygon. 

In [ ]:
from geom.multipolygon_util import get_bounds

In [ ]:
for f in blkgrps['features']:
    f['bounds'] = get_bounds(f)    

In [ ]:
blkgrps['features'][0]['bounds']

Now, we interpolate!

In addition to the pop_cross function, we also import the function called [`point_in_multipolygon`](https://github.com/gisalgs/geom/blob/master/multipolygon_util.py) to help us get determine a more general case. This will import the pip_cross.

In [ ]:
from geom.point import *
from geom.multipolygon_util import point_in_multipolygon

We write a small function that help us determine if a point is outside of a bounding box:

In [ ]:
def out_bound(p, b):
    '''
    p - a Point object
    b - bounds [xmin, xmax, ymin, ymax]
    '''
    return p[0]<b[0] or p[0]>b[1] or p[1]<b[2] or p[1]>b[3]

Now we do a quick interpolation using longitudes and latitudes.

In [ ]:
cells = []
for y in ys:
    for x in xs:
        item = {'point': Point(x, y), 'total':0}
        p = Point(x, y)
        for f in blkgrps['features']:
            if not out_bound(p, f['bounds']):
                if point_in_multipolygon(p, f):
                    item['total'] = f['properties']['Total']
                    break
        cells.append(item)

Now we reorganize the data so that we can display the results. We put the cells in a list of lists where the inner list holds the cells in a row.

In [ ]:
totals = [[] for _ in range(len(ys))]
for i in range(len(ys)):
    totals[len(ys)-i-1] = [c['total'] for c in cells[i*len(xs):(i+1)*len(xs)]]


Then we use [plt.imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) to display it.

In [ ]:
cmap = colormaps.get_cmap('gray_r')
plt.imshow(totals, cmap=cmap)
plt.colorbar(shrink=0.6)

plt.show()

## To be continued in part 2

The coordinates in the above plot seem strange because they are not lat/lon or anything we are familiar with. Do we get everything right? There are two things we need to check before we continue. First, are the ranges correctly cover the county? If not we need to adjust and **redo** the above interpolation. Hint: The ranges should be -83.3 to -82.7 longitudes and 39.6 to 40.2 latitudes.

Second, are we sure we got the geography right? To check that we need to put the county outline on top of the cells and visually confirm. To address this, we will make sure the extent of the plot reflects the coordinates we actually use, and we also want to overlay the county outline to visually confirm. Here is the county outline, another geojson file:

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/refs/heads/master/dissolved_blkgrps.geojson'
with request.urlopen(url) as response:
    franklin = json.loads(response.read())

len(franklin['features'])

Then here is the overlay. (We need to make sure the extent is correct.)

We will force the mapping of the image (i.e., cells) to an extent that defined previously with xmin, xmax, etc. However, to perfectly map the cells, we should know that the cells do not fully fit in the entire extent, from xmin to xmax. Instead, there is a little left on the right side and at the top. So the actual extent is this:

In [ ]:
extent = [xmin, xmin+xn*delta, ymin, ymin+yn*delta]

In [ ]:
_, ax = plt.subplots()

cmap = colormaps.get_cmap('gray_r')
plt.imshow(totals, extent=extent, cmap=cmap)
plt.colorbar(shrink=0.6)

p1 = plt.Polygon(franklin['features'][0]['geometry']['coordinates'][0][0], closed=True, fill=False, edgecolor='blue', alpha=0.5)
ax.add_patch(p1)

plt.show()

We should expect a "perfect" fit!

In sum, the next will involve the following steps:

1. Convert all coordinates to the Ohio South State Plane Coordinate System using the function `spcs_ohio_south` from the [lcc](https://github.com/gisalgs/geom/blob/master/lcc.py) module in geom.
2. Recalculate the population for a new set of cells using the new coordinates.
3. Calculate accessibility for each cell to education services.
4. Draw a lot of maps.


First, we project the block groups and the county outline. To make things simple, we use a function called `proj_multipoly` in the same multipolygon_util module along with the other functions have have used so far (`get_bounds` and `point_in_multipolygon`). This function requires another function to do the actual projection. In our particular case, we are going to use the Ohio South State Plane Coordinate System, which is implemented in the `lcc` module (also in geom and github) and can be imported as follows:

```python
from geom.lcc import spcs_ohio_south
```

The conversion function simply copies everything but the coordinates, which will be converted using the `spcs_ohio_south` function as imported here. 

In our actual implementation of the two-step floating catchment area, we use an "enhanced" version (Luo and Qi 2009) by applying a distance-decay weights in both steps. In short, we have 

$R_j = \Large\frac{S_j}{\sum_{k \in (d_{jk} < d_0)} D_k e^{-(\beta d_{jk})^2}}$

where $e^{-(\beta d_{jk})^2}$ is a general form of a [Gaussian function](https://en.wikipedia.org/wiki/Gaussian_function) and $\beta$ is a parameter. We will see how the use of the parameter changes the result and how to choose one. We also apply weight to the calculation of accessibility

$\Large A_i = \sum_{j\in (d_{ij}<d_0)}{R_j e^{-(\beta d_{ij})^2}}$

The original 2SFCA method is a special case of the enhanced one, which is called E2SFCA.
Work cited:

Luo, W., & Qi, Y. (2009). An enhanced two-step floating catchment area (E2SFCA) method for measuring spatial accessibility to primary care physicians. Health & place, 15(4), 1100-1107.